In [10]:
# Librerie in uso nel codice
import os
import shutil
from pandas import DataFrame

def get_files_data(files, path, map_type):
    '''
    Ottiene name, size, raw_type, type di ogni file.

    files: lista di nome_file.estensione
    path: percorso (str) dove sono situati i file
    map_type: dizionario mappare tipi file a nome. mp3->audio
    '''

    files_data = {'name':[], 'raw_type':[], 'size':[], 'type':[]}

    for file in files:
        file_path = os.path.join(path, file)

        # Controllo file non è una directory
        if os.path.isfile(file_path):

            size = os.path.getsize(file_path)
            name, extens = os.path.splitext(file)
            
            if size and name and extens: # Per essere sicuro che i dati esistono
                files_data['name'].append(name)
                files_data['size'].append(size)
                files_data['raw_type'].append(extens)
                good_type = 'None' # Nel caso non c'è l'estensione del file, append 'None'

                for f_type in map_type.keys():
                    if extens in map_type[f_type]:
                        good_type = f_type
                        break
                files_data['type'].append(good_type)

            else:
                print(f'Missing data for {file}!')

    return files_data # Pandas-style


def move_files(files_data, path, map_dir, verbose=True):
    '''
    Sposta i file in una sub-directory in base al type.

    files_data: dizionario stile Pandas. Ogni key-value pair e' una colonna
    path: percorso (str) dove sono situati i file
    map_dir: dizionario mappare file alla cartella. audio->audios
    verbose: stampa diverse info su ogni file se True
    '''

    for idx in range(len(files_data['name'])): # Accedere ad ogni file
        dir_name = None
        for name in map_dir.keys(): # Ottenere nome sub-directory
            if files_data['type'][idx] in map_dir[name]:
                dir_name = name

        dir_path = path + '/' + dir_name
        if not os.path.exists(dir_path): # Verifica esistenza sub-directory
            os.makedirs(dir_path)
        
        # Sposta il file
        curr_path = path + '/' + files_data['name'][idx] + files_data['raw_type'][idx]
        new_path = dir_path + '/' + files_data['name'][idx] + files_data['raw_type'][idx]
        shutil.move(curr_path, new_path)

        if verbose: # stampa i dati del singolo file
            name = files_data['name'][idx]
            file_type = files_data['type'][idx]
            size = files_data['size'][idx]
            print(f'{name} type:{file_type} size:{size}B')


def get_recap(files_data, path, name='recap'):
    '''
    Crea un file csv con le info dei files.

    files_data: dizionario stile Pandas. Ogni key-value pair e' una colonna
    path: percorso (str) dove si vuole salvare il file csv
    name: nome del file csv senza estensione
    '''

    (DataFrame(files_data)[['name', 'type', 'size']].
        rename(columns={'size':'size (B)'}).
        to_csv(path + '/' + name + '.csv', index=False))


if __name__ == '__main__':
    # Ottiene i file in ordine alfabetico
    path = './files'
    files = sorted(os.listdir(path))

    map_type = {
        'audio': {'.mp3', '.wav', '.flac'},
        'doc': {'.txt', '.odt', '.doc', '.docx', '.pdf'},
        'image': {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg'},
        'video': {'.mp4', '.avi', '.mov', '.mkv', '.wmv'}
    }

    map_dir = {
        'audios': {'audio'},
        'docs': {'doc'},
        'images': {'image'},
        'videos': {'video'}
    }

    files_data = get_files_data(files, path, map_type)
    move_files(files_data, path, map_dir)
    get_recap(files_data, path)

bw type:image size:94926B
ciao type:doc size:12B
daffodil type:image size:24657B
eclipse type:image size:64243B
pippo type:doc size:8299B
song1 type:audio size:1087849B
song2 type:audio size:764176B
trump type:image size:10195B


In [8]:
from pandas import DataFrame

(DataFrame(files_data)[['name', 'type', 'size']].
    rename(columns={'size':'size (B)'}).
    to_csv(path + '/' + 'recap.csv', index=False))